In [ ]:
%cd ..
import glob
import sys
from dmg.realism.mle import whichFitsBetter
from scripts.modelSet import datasets_supported
msetObject = datasets_supported['yakindu-exercise']
train_path = 'data/yakindu-exercise/train'
backend = 'python'
hidden_dim = 64
model_path = 'models/yakindu-exercise-finalModel.m'

# Load dataset

In [ ]:
graphs = [msetObject.getGraphReal(f,backend) 
                for f in glob.glob(train_path + "/*")]

In [ ]:
print('Number of graphs:', len(graphs))

# Nodes and sequence

In [ ]:
import matplotlib.pyplot as plt

h = plt.hist([len(G) for G in graphs], bins = 25)

# Sample from the generator

In [ ]:
from dmg.deeplearning.generativeModel import GenerativeModel
from dmg.deeplearning.generativeModel import sampleGraph
from networkx.algorithms.isomorphism import is_isomorphic
import dmg.graphUtils as gu
import torch


model = GenerativeModel(hidden_dim, msetObject.dic_nodes, 
                        msetObject.dic_edges, 
                        msetObject.operations)
checkpoint = torch.load(model_path,map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
import random
import numpy as np
torch.manual_seed(123)
random.seed(123)
np.random.seed(123)

In [ ]:
number_models = 500
max_size = 200
samples = [sampleGraph(msetObject.pallete.initialGraphs[0], 
                           msetObject.pallete, model, 
                           max_size, msetObject.pallete.separator) 
           for i in range(number_models)]

In [ ]:
inconsistents = []
for s in samples:
    if msetObject.inconsistency(s):
        inconsistents.append(s)
inco_prop = len(inconsistents)*100/len(samples)
not_inconsistents = [g for g in samples if not g in inconsistents]

In [ ]:
inco_prop

In [ ]:
iso = []
for s in samples:
    for g in graphs:
        if (is_isomorphic(s,g,gu.node_match_type, gu.edge_match_type)):
            iso.append(s)
            break
iso_prop = len(iso)*100/len(samples)

In [ ]:
iso_prop

In [ ]:
clean_new_models = [g for g in samples if (not g in inconsistents) and (not g in iso)]
clean_pr = len(clean_new_models)*100/len(samples)
clean_pr

In [ ]:
dot = gu.plotGraphViz(random.sample(clean_new_models,1)[0])
dot